<a href="https://colab.research.google.com/github/ByungjunKim/DDMKL/blob/main/240411_RissScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RISS에서 학위 논문 데이터 수집하기

In [24]:
import requests
import math
import pickle
import time
import sys
import pandas as pd
from tqdm.notebook import tqdm
from random import uniform
import lxml
import lxml.etree as et
from bs4 import BeautifulSoup
import glob
import urllib3
urllib3.disable_warnings()
from natsort import natsorted
import re
import concurrent.futures

In [25]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "DNT": "1", # Do Not Track 요청 헤더 (사용자 추적 거부)
    # 필요한 추가 헤더를 여기에 추가
}

In [26]:
def request_until_success(url, headers, timeout=5, delay=2):
    """
    Continuously makes a request to the specified URL with a timeout until the request is successful.

    Args:
    - url (str): The URL to request.
    - timeout (int): The timeout for the request in seconds.
    - delay (int): The delay between retries in seconds.

    Returns:
    - response: The successful response from the server.
    """
    attempt = 1
    while True:
        try:
            # print(f"Attempt {attempt}")
            response = requests.get(url,headers=headers, timeout=timeout)
            response.raise_for_status()  # Raises HTTPError for bad responses
            # print("Request successful")
            return response
        except requests.Timeout as e:
            print(f"Request timed out: {e}")
        except requests.RequestException as e:
            print(f"Request failed: {e}")

        # print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        attempt += 1

### 검색 조건에 맞는 URL 설정
학과정보 : 국어국문 OR 국문  
학위수여연도 : 2000~2023  
학위유형 : 국내박사  
https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=

In [27]:
url = 'https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query='

In [28]:
req = request_until_success(url,headers).text

In [29]:
soup = BeautifulSoup(req, 'lxml')

In [30]:
cont_ml60_classes = soup.find_all('div', class_='cont ml60')

data = [{
    'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
    'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
    'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
    'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
    'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
    'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
    'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
} for cont_ml60 in cont_ml60_classes]

In [31]:
data

[{'title': '외국인 유학생 대상국어국문학 전공 기초 교육과정 개발 연구',
  'link': '/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=3999e8d77c9617e7ffe0bdc3ef48d419&keyword=',
  'writer': '임형옥',
  'assigned': '고려대학교 대학원',
  'year': '2020',
  'grad': '국내박사',
  'preAbstract': 'The aim of this study is to develop a preparation curriculum for Korean language and literature major international students. International students who study in Korean universities often have difficulties when they take core subjects, mainly due to lack of essential knowledge in their major or incompetent task performance. If it is the case, a major preparation curriculum may reduce such obstacles and help the students feel confident in their major subjects. In fact, majoring in the Korean language and literature needs far higher command of Korean than others, which implies that international students majoring in the Korean language and literature require a high level of Korean proficiency and competence in specific lan

### 기본 정보 자동으로 수집하기

In [32]:
# Assuming 'soup' is your BeautifulSoup object
num_span = soup.find('span', class_='num')

if num_span is not None:
    total_count = int(num_span.get_text().replace(',', ''))
else:
    total_count = 0

total_count

4811

In [ ]:
# # The base URL without the 'iStartCount' parameter
# base_url = "https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor,국어국문@op,OR@znMajor,국문&strQuery=&exQuery=&exQueryText=&order=/DESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="

# # Calculate the number of iterations
# iterations = math.ceil(total_count / 100)

# # Initialize an empty list to store the data
# data = []

# # Loop over the range, incrementing by 100 each time
# for i in tqdm(range(iterations)):
#     # Create the URL with the incremented 'iStartCount' parameter
#     url = f"{base_url}&iStartCount={i * 100}"

#     # Send a GET request to the URL
#     # response = requests.get(url)
#     response = request_until_success(url, headers)

#     # Create a BeautifulSoup object from the response text
#     soup = BeautifulSoup(response.text, 'html.parser')

#     # Find all 'div' elements with class 'cont ml60'
#     cont_ml60_classes = soup.find_all('div', class_='cont ml60')

#     # Extract the data and append it to the list
#     data.extend([{
#         'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
#         'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
#         'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
#         'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
#         'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
#         'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
#         'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
#     } for cont_ml60 in cont_ml60_classes])

In [33]:
# The base URL without the 'iStartCount' parameter
base_url = "https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor,국어국문@op,OR@znMajor,국문&strQuery=&exQuery=&exQueryText=&order=/DESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="

# Calculate the number of iterations
iterations = math.ceil(total_count / 100)

# Initialize an empty list to store the data
data = []

def fetch_data(i):
    # Create the URL with the incremented 'iStartCount' parameter
    url = f"{base_url}&iStartCount={i * 100}"

    # Send a GET request to the URL
    response = request_until_success(url, headers)

    # Create a BeautifulSoup object from the response text
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all 'div' elements with class 'cont ml60'
    cont_ml60_classes = soup.find_all('div', class_='cont ml60')

    # Extract the data and return it
    return [{
        'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
        'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
        'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
        'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
        'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
        'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
        'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
    } for cont_ml60 in cont_ml60_classes]

# Loop over the range, incrementing by 100 each time
with concurrent.futures.ThreadPoolExecutor() as executor:
    data_list = list(tqdm(executor.map(fetch_data, range(iterations)), total=iterations))

# Flatten the list of lists
data = [item for sublist in data_list for item in sublist]

  0%|          | 0/49 [00:00<?, ?it/s]

In [35]:
df = pd.DataFrame(data)
df

,title,link,writer,assigned,year,grad,preAbstract
0,외국인 유학생 대상국어국문학 전공 기초 교육과정 개발 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,임형옥,고려대학교 대학원,2020,국내박사,The aim of this study is to develop a preparat...
1,중국 내 한국어학과 문학 교육과정의 설계와 재구성 연구 : 길림성 소재 대학의 고전...,/search/detail/DetailView.do?p_mat_type=be54d9...,정반반,공주대학교 일반대학원,2021,국내박사,이 글은 중국 길림성 소재 대학의 한국어학과 문학교육과정에 관한 내용이다. 1972...
2,인문계열 학과 법제 교육 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,김태림,경희대학교 대학원,2023,국내박사,The purpose of this study is to emphasize the ...
3,러시아에서의 한국문학 수용 과정 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,한승희,한양대학교 대학원,2021,국내박사,None
4,한국 근대시 형성과 漢文脈,/search/detail/DetailView.do?p_mat_type=be54d9...,정기인,서울대학교,2017,국내박사,본 논문은 한국 근대시 형성에 기여한 주요 문인들이 한문맥(漢文脈)의 자원을 활용하...
...,...,...,...,...,...,...,...
4806,魯迅 雜文의 思惟 樣相 硏究 : 認識과 表現의 問題를 中心으로,/search/detail/DetailView.do?p_mat_type=be54d9...,고점복,고려대학교 대학원,2007,국내박사,본고는 魯迅의 雜文을 통해 그의 思惟 특징을 밝히고자 하였다. 雜文쓰기는 그의 창작...
4807,중국의 탈놀이 <<大頭和尙>> 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,김순희,成均館大學校,2005,국내박사,None
4808,沈從文 後期小說의 敍述者 硏究,/search/detail/DetailView.do?p_mat_type=be54d9...,임향섭,단국대학교 대학원,2005,국내박사,None
4809,중국 향토 서정소설의 서사미학 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,WANG JUAN,고려대학교 대학원,2023,국내박사,A critical study of local lyrical novels start...


### 논문별 상세정보 수집하기 (작업 중)

In [36]:
df['link'] = 'https://www.riss.kr' + df['link']
df['link']

0       https://www.riss.kr/search/detail/DetailView.d...
1       https://www.riss.kr/search/detail/DetailView.d...
2       https://www.riss.kr/search/detail/DetailView.d...
3       https://www.riss.kr/search/detail/DetailView.d...
4       https://www.riss.kr/search/detail/DetailView.d...
                              ...                        
4806    https://www.riss.kr/search/detail/DetailView.d...
4807    https://www.riss.kr/search/detail/DetailView.d...
4808    https://www.riss.kr/search/detail/DetailView.d...
4809    https://www.riss.kr/search/detail/DetailView.d...
4810    https://www.riss.kr/search/detail/DetailView.d...
Name: link, Length: 4811, dtype: object

In [37]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Cookie": "Your Cookie Here",
    "Host": "www.riss.kr",
    "Referer": df['link'][0],
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows"
}

# response = requests.get(df['link'][0], headers=headers)
# soup = BeautifulSoup(response.text, 'lxml')

In [ ]:
# data_list = []

# for link in tqdm(df['link']):
#     headers["Referer"] = link
#     response = request_until_success(link, headers=headers)
#     soup = BeautifulSoup(response.text, 'lxml')
#     data = {}
#     data['link'] = link
#     for li in soup.find('div',class_='infoDetailL').find_all('li'):
#         key_element = li.find('span', {'class': 'strong'})
#         value_element = li.find('div')
#         if key_element and value_element:
#             key = key_element.text.strip()
#             value = value_element.text.strip()
#             data[key] = value
#     text_off_divs = soup.find('div', class_='content additionalInfo').find_all('div', class_='text off')

#     title_text_dict = {}

#     for div in text_off_divs:
#         title = div.find_previous_sibling('p', class_='title')
#         if title:
#             if title.text.strip() == "참고문헌 (Reference)":
#                 title_text_dict[title.text.strip()] = [p.text.strip() for p in div.find_all('p')]
#             else:
#                 title_text_dict[title.text.strip()] = div.text.strip()
#     data.update(title_text_dict)
#     data_list.append(data)

In [ ]:
def fetch_data(link):
    headers["Referer"] = link
    # response = requests.get(link, headers=headers)
    response = request_until_success(link, headers)
    soup = BeautifulSoup(response.text, 'lxml')
    data = {}
    data['link'] = link
    for li in soup.find('div',class_='infoDetailL').find_all('li'):
        key_element = li.find('span', {'class': 'strong'})
        value_element = li.find('div')
        if key_element and value_element:
            key = key_element.text.strip()
            value = value_element.text.strip()
            data[key] = value
    try:
        text_off_divs = soup.find('div', class_='content additionalInfo').find_all('div', class_='text off')
        title_text_dict = {}
        for div in text_off_divs:
            title = div.find_previous_sibling('p', class_='title')
            if title:
                if title.text.strip() == "참고문헌 (Reference)":
                    title_text_dict[title.text.strip()] = [p.text.strip() for p in div.find_all('p')]
                else:
                    title_text_dict[title.text.strip()] = div.text.strip()
        data.update(title_text_dict)
    except:
        print(link)
    return data

with concurrent.futures.ThreadPoolExecutor() as executor:
    data_list = list(tqdm(executor.map(fetch_data, df['link'].tolist()), total=len(df['link'])))

  0%|          | 0/800 [00:00<?, ?it/s]

https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=c458192d34e99581&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=095568c062bab1d8&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=13d75466b42e6f7d&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=e593629b6a8f9ca5&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=e7dd99d4fbd744de&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=a31319f6bb2ec798ffe0bdc3ef48d419&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=f889a4b7049837c4&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=23c6d352e737177effe0bdc3ef48d419&keyword=
https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d

In [ ]:
detail = pd.DataFrame(data_list)
detail

,link,저자,발행사항,학위논문사항,발행연도,작성언어,주제어,발행국(도시),기타서명,형태사항,일반주기명,UCI식별코드,DOI식별코드,소장기관,다국어 초록 (Multilingual Abstract),목차 (Table of Contents),참고문헌 (Reference),DDC,국문 초록 (Abstract),KDC
0,https://www.riss.kr/search/detail/DetailView.d...,임형옥,"서울 : 고려대학교 대학원, 2020",학위논문(박사)-- 고려대학교 대학원: 국어국문학과 2020. 8,2020,한국어,특정 학문 목적 한국어교육\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t...,서울,A study on the development of a preparation cu...,"vii, 239 p. ; 26 cm",부록수록지도교수: 김정숙참고문헌: p. 205-214,I804:11009-000000231531,10.23186/korea.000000231531.11009.0001168,고려대학교 도서관,The aim of this study is to develop a preparat...,목 차\n\n1. 서론1\n1.1. 연구의 목적 및 필요성1\n1.2. 선행연구 검...,"[1 이익섭, ""?국어학개설?"", 학연사, 2011, 2 신승용, ""「국어학 개...",NaN,NaN,NaN
1,https://www.riss.kr/search/detail/DetailView.d...,정반반\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\...,"공주 : 공주대학교 일반대학원, 2021",학위논문(박사)-- 공주대학교 일반대학원 : 국어국문학과 2022. 2,2021,한국어,"중국 한국어학과, 한국문학교육과정, 고전문학",충청남도,A Study on the Design and Reconstruction of Ko...,183p. 26cm,지도교수:권대광참고문헌 : p.164-170,I804:44004-000000031401,NaN,국립공주대학교 도서관,NaN,I. 서론 1\n 1. 연구의 필요성 및 목적 1\n 2. 선행연구 검토 5\n 3...,NaN,811 판사항(22),이 글은 중국 길림성 소재 대학의 한국어학과 문학교육과정에 관한 내용이다. 1972...,NaN
2,https://www.riss.kr/search/detail/DetailView.d...,김태림,"서울 : 경희대학교 대학원, 2023",학위논문(박사)-- 경희대학교 대학원 : 국어국문학과 2023. 2,2023,한국어,"인문학, 국어국문학, 법제 교육, 저작권, 지식재산권",서울,A Study on Legal Education of Humanities Depar...,"vii, 122 p. : 삽화 ; 26 cm",경희대학교 논문은 저작권에 의해 보호받습니다.지도교수: 이정재참고문헌: p. 108...,I804:11006-200000659401,NaN,경희대학교 중앙도서관,The purpose of this study is to emphasize the ...,<국문초록>ⅰ\n\nⅠ. 서론 1\n제1절 연구의 필요성 및 목적 1\n제2절 연구...,NaN,811 판사항(22),「인문계열 학과 법제 교육 연구」는 인문학 학습자들이 사회 현장에서 겪는 실제적인 ...,NaN
3,https://www.riss.kr/search/detail/DetailView.d...,한승희,"서울 : 한양대학교 대학원, 2021",학위논문(박사)-- 한양대학교 대학원 : 국어국문학과 2021. 2,2021,한국어,한국문학,서울,NaN,"ii, 174 p. : 삽도 ; 26 cm.","권두 국문요지, 권말 Abstract 수록지도교수: 유성호참고문헌: p. 156-172",I804:11062-200000485955,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,[ 목차 ]\n\n\n국문요지\t Ⅱ\n\n제1장 서론\t 1\n제1절 연구목적과 ...,NaN,NaN,NaN,NaN
4,https://www.riss.kr/search/detail/DetailView.d...,정기인,"서울 : 서울대학교, 2017",학위논문(박사) -- 서울대학교 대학원 국어국문학과 현대문학전공 2017,2017,한국어,한문맥\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\r\n\t\...,서울,NaN,"viii, 301 p. ; 26 cm",지도교수: 김유중참고문헌: p. 276-294,NaN,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,I. 서론\n 1. 연구사 검토 및 문제 제기\n 2. 연구의 시각 및 대상\n...,NaN,895.7 판사항(23),본 논문은 한국 근대시 형성에 기여한 주요 문인들이 한문맥(漢文脈)의 자원을 활용하...,810.906 판사항(6)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.riss.kr/search/detail/DetailView.d...,김점숙,"서울 : 한양대학교 대학원, 2014",학위논문(박사)-- 한양대학교 대학원 : 국어국문학과 2014. 2,2014,한국어,한국문학,서울,NaN,"v, 206 p. : 삽도 ; 26 cm.","지도교수: 이재복권두 국문요지, 권말 Abstract 수록참고문헌: p. 195-202",NaN,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,차 례\r\n \n<국문요지> \t ⅲ\r\n \nⅠ. 서론\t 1\r\n ...,NaN,NaN,국문요지 김춘수 시의 특징 중 하나는 자신의 시세계를 끊임없이 부정한다는 점이다....,NaN
96,https://www.riss.kr/search/detail/DetailView.d...,변이주,"군산 : 군산대학교, 2014",학위논문(박사)-- 군산대학교 : 국어국문학과 2014. 8,2014,한국어,NaN,전북특별자치도,NaN,; 26 cm,NaN,NaN,NaN,군산대학교 도서관,The purpose of this study is to identify the s...,NaN,NaN,NaN,본 연구의 목적은 개역성경의 개정판들에 쓰인 우리말 중 국어정서법을 기준으로 하여 ...,NaN
97,https://www.riss.kr/search/detail/DetailView.d...,이병철,"서울 : 한양대학교 대학원, 2019",학위논문(박사)-- 한양대학교 대학원 : 국어국문학과 2019. 2,2019,한국어,한국문학,서울,NaN,173 p. : 삽도 ; 26 cm.,"권두 국문요지, 권말 Abstract 수록지도교수: 유성호설명적 각주 수록참고문헌:...",I804:11062-000000108354,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,□ 국문요지\n\nⅠ. 서론\t1\n1. 연구 목적\t1\n2. 선행 연구 검토\t...,"[1 서정주, ""신라초"", 정음사, 1961, 2 서정주, ""화사집"", 남만서고...",NaN,서정주의 첫 시집인 『화사집』은 에로스와 리비도로 충만한 육체적 생명성의 이미지들로...,NaN
98,https://www.riss.kr/search/detail/DetailView.d...,김수애\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\...,"제주 : 제주대학교, 2020",학위논문(박사)-- 제주대학교 대학원 : 국어국문학과 2020. 8,2020,한국어,언간(한글편지)\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\r...,제주특별자치도,A Korean Linguistic Study on Queen Sunwon’s Eo...,"vi,191p : 삽화 ; 30 cm",지도교수: 배영환참고문헌 : p.139-149,I804:49002-000000009577,NaN